### Setup code

In [115]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from matplotlib import image
import seaborn as sns
import re
%config InlineBackend.figure_format = 'retina'


### Executive Summary

The main aim of this project is to develop a dog breed image classifier, principally through neural network methodologies.

The further aim is to evaluate the landscape of freely available frameworks and tutorials to see whether it is possible for a novice data scientist to achieve results in a new field of expertise.

Models deployed were split between non-neural networks (to provide a benchmark of what neural networks should be able to achieve) and neural networks of increasing sophistication along with associated techniques for model improvement.

The success of the project will be measured by:
- the accuracy of the dog breed image classifier model
- the complexity of the dog breed image classifier model achieved through open source frameworks and tutorials.

### Goals

The goal of this project is to explore the performance of neural network image classifiers through the development of a dog breed image classifier using a real-world dataset.

As part of the data science immersive, I have observed the following:
- the ability to use basic modelling syntax and structure and apply that to different model types
- the wide variety of modelling infrastructure freely available
- the large amounts of online community support to assist in model implementation

I want to use this project to explore these observations as applied to complex models. I would like to do this to see how much success can be obtained applying complex models particularly to a real-world dataset. There seems to be a bias to using pre-cleaned data (e.g. from Kaggle competitions) in model tutorials which results in artifically high model performance. I wanted to see if high model performance could be achieved on a dataset scraped from real-world pictures.

The goals of the project are therefore two-fold:
- develop as high-performing dog breed image classifier as possible using real-world dataset.
- following the observations above, assess the current landscape of freely available neural network modelling methods and their ease of implementation

Metrics for success will therefore be split between the two goals:
- develop a model which scores well on classifying dog breeds from images
- successful implementation of a variety of neural network models and techniques

### Modelling choices

Modelling approaches were split into two main areas - standard modelling approaches and neural networks. Standard modelling approaches were used to set an effective benchmark for what the neural networks should be achieving.

Standard modelling approaches included:
- Logistic Regression
- Logistic Regression CV
- Random Forest Classifier (RFC)
- Naive Bayes
- Gradient Boosting Classifier (GBC)
- Support Vector Machines (SVM)

Grid Search was used to tune parameters/kernels (where applicable) for RFC, GBC and SVM. 

Principle Component Analysis was initially used in the case of 3 breeds to reduce the number of features from 480,000 (400 x 400 x 3) to the number of images in the sample. In fact when using more than 3 breeds the PCA process failed to run on my computer and hence the resolution of images was reduced to (128 x 128).

PCA results were successful though - below shows the scree plot and how 480,000 features could be reduced to only 1,200 with little loss in detail.

<img src="report_images/scree_plot.png" align="left"/>

Neural network approaches included:
- Building up sophistication of models with increasing number of layers. See Appendix for more on the different model approaches used.
- Early stopping call backs
- Introducing drop-out
- Image augmentation - rotation range of 90 degrees, and random horizontal flips - see example here:

<img src="clean_dog_pics/00003_pomchi_34.jpg" align="left"/>

<img src="report_images/image_augmentation_example.jpg" align="left"/>

- Transfer learning - this involved taking pre-trained VGG16 and ResNet50 models and freezing all layers apart from the final one, which was then trained on this dataset.

The dataset used for modelling was scraped from borrowmydoggy.com, a website which matches owners of dogs with people wanting to spend time with dogs. Each dog profile on the website has pictures and the breed of the dog. I scraped around 32,000 pictures along with their breeds.

Then for each model run, the dataset used was determined as follows:

- choice of breeds used for modelling. Given the number of breeds in the dataset, I simplified modelling initially to only using 3 different breeds but then later expanded to test model effectiveness.
- whether the pictures were cleaned or not. Cleaning process removed the background from pictures with idea being that this should improve model accuracy. Example:

<img src="report_images/summary_image_origvsclean.jpg" align="left"/>

- threshold for image quality was initially set at 30%. Image quality is defined as % of pixels in the image which belong to the dog. If the dog is smaller in the picture then the image quality % will be lower, if it's higher then the measure will be higher. 30% was set to exclude bad pictures which may cause problems for the models. Impact of threshold will be investigated later on.

### Model performance

#### Standard model performance

##### First breed selection: Labrador, Cocker Spaniel and French Bulldog

These three breeds were chosen as they had some of the highest number of pictures. 

Size of image set was 2,589, with a train/test split of 20%. Due to some small class imbalance, the baseline score was 0.405 with a total train image size of 2,589 images. % split of train images 41% Labrador; 37% Cocker Spaniel; 22% French Bulldog.

Sample pictures here:

<img src="report_images/breed1_sample_pics.jpg" align="left"/>

With original pictures, best validation scores were achieved by the support vector machines. This was c. 10% ahead of the baseline, generalising well to the test images as well.

<img src="report_images/std_model_perf_b1_original.jpg" align="left"/>

The cleaned pictures showed some improvement over the original pictures, notably in the case of the support vector machines. This increased the mean validation score for the SVM to over 16% higher than the baseline.

<img src="report_images/std_model_perf_b1_origvsclean.jpg" align="left"/>

Bringing in neural network models now, results are mixed. 

For the original pictures, all but one (NN model 4 + IA) of the models fail to improve validation scores significantly above the base line score.

For the cleaned pictures, the results are more successful, but only one model (NN model 4) outscores the SVC validation score significantly.

Volatility in the results is also highlighted by the contrast between the NN model 3 & 4 with image augmentation.

<img src="report_images/nn_model_perf_b1_origvsclean.jpg" align="left"/>

##### Second breed selection: Staffordshire Bull Terrier, Jack Russel Terrier, Border Collie

These three breeds were again chosen as they had comparatively high number of pictures. 

Size of image set was 1,496, with a train/test split of 20%. Due to some small class imbalance, the baseline score was 0.383. % split of train images 38% Staffordshire Bull Terrier; 34% Jack Russell Terrier; 28% Border Collie.

Sample pictures here:

<img src="report_images/breed2_sample_pics.jpg" align="left" />

Similar results were achieved to the first set of breeds. The Support Vector Machine scored the highest out of the non-neural network models on the original pictures.

<img src="report_images/std_model_perf_b2_original.jpg" align="left"/>

The cleaned pictures gave a few percentage point increase in model results for all non-neural network methods used.

<img src="report_images/std_model_perf_b2_origvsclean.jpg" align="left"/>

The neural network models again failed to consistently provide a significant increase vs the SVM method, although image augmentation did help model 4 improve above the SVM.

The volatility in the neural network results is again striking.

<img src="report_images/nn_model_perf_b2_origvsclean.jpg" align="left"/>

##### Third breed selection: previous 6 breeds combined

With the first 6 breeds combined, the baseline score dropped to 0.257. There was some class imbalance but not significant - 25.7% of pictures were of Labradors, whereas only 10% were of Border Collies.

As mentioned previously, the PCA process for non-neural network models could not function at full resolution hence picture resolution was reduced from 400x400 to 128x128. These were used for both non-neural network models and neural network models to allow comparability between the modelling techniques.

There was a similar story to modelling the breeds in two separate groups though. Again, the Support Vector Machine algorithm worked best out of the non-neural network models.

<img src="report_images/std_model_perf_b3_original.jpg" align="left"/>

Again, the cleaned pictures produced a few percentage points increase in accuracy in most cases.

<img src="report_images/std_model_perf_b3_origvsclean.jpg" align="left"/>

This time however, the neural network models failed to converge at all. Only the simplest model scored higher than the baseline, and even then lower than the SVM model.

<img src="report_images/nn_model_perf_b3_clean.jpg" align="left"/>

#### Investigations into factors affecting neural network model performance

##### Number of images in the training set

The number of images available has a strong impact on neural network model performance.

With fewer than 250 pictures, the model fails to achieve validation scores higher than the base line of 0.40.

Validation scores improve when the number of pictures is around 500. While scores improve subsequently, the high score at around 750 images suggest model stability has not been achieved.

The continuing increase even up to the limit of this dataset shows that more data would likely result in stronger model performance.



<img src="report_images/number_of_images_impact.jpg" align="left"/>

##### Resolution of images on model accuracy

The results of investigation into impact of image resolution were somewhat counterintuitive. The below chart has x-axis representing the square resolution of each picture e.g. x = 50 => image resolution = 50 by 50. Conclusions are:
- increasing resolution generally reduces validation scores
- neural network validation scores are volatile

<img src="report_images/resolution_of_images_impact.jpg" align="left"/>

##### Quality of images on model accuracy

The 'quality' of images refers to the percentage of pixels in the image showing the dog. The higher the score, the bigger the dog would appear in the picture.

The below chart has the x-axis representing the threshold of image quality for a dataset. For x=20, this would mean that no image with image quality below 20% was included in the training or test set.

The results here show that as the quality of images threshold increases, the results become more stable 

<img src="report_images/image_quality_impact.jpg" align="left"/>

### Results

#### Goal 1 - build classification model

From the above results, two key findings arise:
1. The neural network models scored the highest scores when classifying low numbers of breeds.
2. The support vector machine models were the most consistent and look like they generalise better to higher numbers of breeds.


#### Goal 2 - successful implementation of a variety of neural network models and techniques

Of the neural network techniques I employed, there were different levels of success. Increasing the layers, employing early stopping mechanisms, adding dropouts into the layers and image augmentation were more successful.

While I have shown examples of some of the result from the modelling process above, during the investigations I performed more modelling and the impression I have overall is that particularly image augmentation led to more robust results. Simpler models were generally overfitting very quickly in earlier epochs, with the train scores quickly exceeding the validation scores.

<img src="report_images/clean_NN_model_2_overfit_epoch_history.jpg" align="left"/>

By contrast, the validation scores increased more in line as epochs progressed once the techniques above were employed, particularly image augmentation. It may have taken more time to run but results were more consistent.

<img src="report_images/clean_NN_model_3_breeds_1_epoch_history.jpg" align="left"/>



I was particularly pleased in getting this to work as it required learning a whole new method of providing images to the model using image generators which was not intuitive. 

### Conclusions

In terms of model performance, based on what I was able to implement, I would recommend the following approach:
- use support vector machines to establish a baseline for what standardised models can do
- implement neural networks with early stopping, dropouts and image augmentation to see if any higher scores can be achieved
- source more data to improve model scores

If model scores could be improved high enough, then this model could be used in commercial application by borrowmydoggy.com in two ways:
1. When users upload dog photos, the model could automatically fill in which breed the dog is rather than have the user type it in.
2. The picture cleaning process which quantifies how many pixels in a picture relate to a dog could be used to give advice to users who upload a picture which does not show their dog clearly enough.

If I had more time I would:
- address class imbalance in samples
- look at variance of cross-validation scores to review model stability
- explore model performance with different PCA features
- implement fully pre-trained model to recognise breeds it had been originally trained on.

### Appendix

#### Neural network model code

##### Model 1

In [ ]:
#https://www.tensorflow.org/beta/tutorials/keras/basic_classification
def def_model1(outshape):
    model = models.Sequential([layers.Flatten(input_shape=(400, 400, 3)),
                               layers.Dense(128, activation='relu'),
                               layers.Dense(outshape, activation='softmax')])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

##### Model 2

In [ ]:
#https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-to-classify-satellite-photos-of-the-amazon-rainforest/
def def_model2(outshape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(400, 400, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(outshape, activation='softmax'))

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model

##### Model 3

In [ ]:
#Set up third model definition to include dropouts and early stopping.
def def_model3(outshape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(400, 400, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(outshape, activation='softmax'))

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    es = EarlyStopping(monitor = 'val_acc', min_delta = 0.0001, restore_best_weights=True)
    return model, es

##### Model 4

In [ ]:
#Set up fourth model definition to include a further 2 layers.
def def_model4(outshape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(400, 400, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(outshape, activation='softmax'))

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    es = EarlyStopping(monitor = 'val_acc', min_delta = 0.0001, restore_best_weights=True)
    return model, es